# Neural Network

The purpose of this notebook is to train a neural newtork to classify spoken words. It starts with the VGG 16 pre-trained weights. It then one by one unlocks the 16 layers to retrain the weights.

In [1]:
# Feature extraction example
import numpy as np
import librosa
from librosa import display
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import os
import glob
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


# Load data, features

In [2]:
def encode(val):
    values = np.array(val)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    encoded = np_utils.to_categorical(integer_encoded).astype(int)
    return encoded

## Load Features without Background Noise

In [3]:
car = ['stop','two','three','happy','left','right']
with open('No_Background_Files/go_features.pkl', 'rb') as f:
    data = pickle.load(f)
    feats = data
with open('No_Background_Files/go_labels.pkl', 'rb') as f:
    data1 = pickle.load(f)
    labs = data1
for cmd in car:
    with open('No_Background_Files/'+str(cmd) +'_features.pkl', 'rb') as f:
        data = pickle.load(f)
        feats = np.concatenate((feats,data))
    with open('No_Background_Files/'+str(cmd) +'_labels.pkl', 'rb') as f:
        data1 = pickle.load(f)
        labs = np.concatenate((labs,data1))

## Load Unknowns

In [49]:
unknowns = ['bird','cat','dog','bird','one','two','three','four','five','six'
           ,'seven','eight','nine','zero','bird','happy','house','marvin','sheila'
           ,'tree','wow']
with open('No_Background_Files/unknowns/bedfeatures.pkl', 'rb') as f:
    data = pickle.load(f)
    unknown = data
with open('No_Background_Files/unknowns/bedlabels.pkl', 'rb') as f:
    data1 = pickle.load(f)
    unknowns_labs = data1
for cmd in unknowns:
    with open('No_Background_Files/unknowns/'+str(cmd) +'features.pkl', 'rb') as f:
        data = pickle.load(f)
        unknown = np.concatenate((unknown,data))
    with open('No_Background_Files/unknowns/'+str(cmd) +'labels.pkl', 'rb') as f:
        data1 = pickle.load(f)
        unknowns_labs = np.concatenate((unknowns_labs,data1))

## Make inputs the correct shape

In [4]:
features = np.concatenate((feats, np.zeros(np.shape(feats))), axis = 3)
features = np.concatenate((features, np.zeros(np.shape(feats))), axis = 3)
features = np.pad(features, [(0, 0),(0,0), (0, 7),(0,0)], mode='constant', constant_values=0)

In [5]:
features.shape

(14962, 128, 48, 3)

### Train/Val Split

In [6]:
X_train_int, X_test, y_train_int, y_test = train_test_split(features,labs,test_size=0.2,random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train_int,y_train_int,test_size=0.2,random_state=2)

In [7]:
import pickle
filename = 'X_train.pkl'
pickle.dump(X_train, open(filename, 'wb'))

filename = 'y_train.pkl'
pickle.dump(y_train, open(filename, 'wb'))

filename = 'X_test.pkl'
pickle.dump(X_train, open(filename, 'wb'))

filename = 'y_test.pkl'
pickle.dump(y_test, open(filename, 'wb'))

### VGG16 Trained CNN

In [8]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint,EarlyStopping
# note we exclude the final dense layers and add one back below, we would retrain it ourselves
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 48, 3)) 
 
# Freeze convolutional layers
# for layer in base_model.layers[-4:]:
#     layer.trainable = True    
for layer in base_model.layers:
    layer.trainable = False 
    
x = base_model.output
x = Flatten()(x) # flatten from convolution tensor output 

densee0 = Dense(1024, activation='relu')(x)
dd0 = Dropout(.23)(densee0)
densee1 = Dense(512, activation='relu')(dd0)
dd1 = Dropout(.23)(densee1)
densee2 = Dense(256, activation='relu')(dd1)
d0 = Dropout(.23)(densee2)
dense1 = Dense(128, activation='relu')(d0)
d1 = Dropout(.23)(dense1)
dense2 = Dense(128, activation='relu')(d1)
d2 = Dropout(.23)(dense2)
dense3 = Dense(64, activation='relu')(d2)
d3 = Dropout(.23)(dense3)
dense4 = Dense(32, activation='relu')(d3)
d4 = Dropout(.23)(dense4)
predictions = Dense(7, activation='softmax')(d4) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
lr=.001
optimizer = Adam()
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
y_train_cat = encode(y_train)
y_val_cat = encode(y_val)
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
model.fit(X_train, y_train_cat, batch_size=64, epochs=100, validation_data=(X_val, y_val_cat),callbacks=callbacks)

Train on 9575 samples, validate on 2394 samples
Epoch 1/100
9575/9575 [==============================] - 24s 2ms/step - loss: 1.9851 - acc: 0.1490 - val_loss: 1.9420 - val_acc: 0.1345
Epoch 2/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.9337 - acc: 0.1670 - val_loss: 1.8536 - val_acc: 0.2210
Epoch 3/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.8360 - acc: 0.2319 - val_loss: 1.7031 - val_acc: 0.2602
Epoch 4/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.7444 - acc: 0.2632 - val_loss: 1.6731 - val_acc: 0.2870
Epoch 5/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.7003 - acc: 0.2821 - val_loss: 1.6263 - val_acc: 0.2870
Epoch 6/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.6805 - acc: 0.2890 - val_loss: 1.6160 - val_acc: 0.2999
Epoch 7/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.6501 - acc: 0.2916 - val_loss: 1.6017 - val_acc: 0.3124


In [12]:
def release_layers(i,lr):
    for layer in base_model.layers[i:]:
        layer.trainable = True
    optimizer = Adam(lr=lr)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    print("Layer: "+str(i))
    callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
    model.fit(X_train, y_train_cat, batch_size=64, epochs=100, validation_data=(X_val, y_val_cat),callbacks=callbacks)

In [13]:
w = list(range(-1,-20,-1))

lr=.0001
for i in w:
    release_layers(i,lr)


#release_layers(-20,.0001)

Layer: -1
Train on 9575 samples, validate on 2394 samples
Epoch 1/100
9575/9575 [==============================] - 18s 2ms/step - loss: 1.2363 - acc: 0.4960 - val_loss: 1.2940 - val_acc: 0.5084
Epoch 2/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.2106 - acc: 0.5062 - val_loss: 1.2923 - val_acc: 0.5201
Epoch 3/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.2024 - acc: 0.5039 - val_loss: 1.2853 - val_acc: 0.5230
Epoch 4/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.1982 - acc: 0.5095 - val_loss: 1.2918 - val_acc: 0.5272
Epoch 5/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.1748 - acc: 0.5191 - val_loss: 1.2834 - val_acc: 0.5167
Epoch 6/100
9575/9575 [==============================] - 18s 2ms/step - loss: 1.1874 - acc: 0.5161 - val_loss: 1.2863 - val_acc: 0.5221
Epoch 7/100
9575/9575 [==============================] - 17s 2ms/step - loss: 1.1756 - acc: 0.5210 - val_loss: 1.2863 - val_ac

9575/9575 [==============================] - 44s 5ms/step - loss: 0.1149 - acc: 0.9676 - val_loss: 0.2931 - val_acc: 0.9348
Epoch 3/100
9575/9575 [==============================] - 43s 5ms/step - loss: 0.0920 - acc: 0.9722 - val_loss: 0.2304 - val_acc: 0.9553
Epoch 4/100
9575/9575 [==============================] - 44s 5ms/step - loss: 0.0873 - acc: 0.9745 - val_loss: 0.2562 - val_acc: 0.9524
Epoch 5/100
9575/9575 [==============================] - 44s 5ms/step - loss: 0.0715 - acc: 0.9786 - val_loss: 0.3605 - val_acc: 0.9461
Layer: -19
Train on 9575 samples, validate on 2394 samples
Epoch 1/100
9575/9575 [==============================] - 48s 5ms/step - loss: 0.1391 - acc: 0.9675 - val_loss: 0.2191 - val_acc: 0.9382
Epoch 2/100
9575/9575 [==============================] - 43s 4ms/step - loss: 0.1718 - acc: 0.9587 - val_loss: 0.2778 - val_acc: 0.9403
Epoch 3/100
9575/9575 [==============================] - 44s 5ms/step - loss: 0.0719 - acc: 0.9791 - val_loss: 0.2568 - val_acc: 0.9536


## Save Model

In [14]:
# serialize model to JSON
model_json = model.to_json()
with open("model_car_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_car_final.h5")
print("Saved model to disk")


# load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")

Saved model to disk


In [8]:
from keras.models import model_from_json
#load json and create model
json_file = open('model_car_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_car_final.h5")
print("Loaded model from disk")

Loaded model from disk


## Test Models

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

label_encoder = LabelEncoder()
y_pred = loaded_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=-1)
y_test_encoded = label_encoder.fit_transform(y_test)
accuracy_score(y_test_encoded, y_pred_classes)
#confusion_matrix(y_test_encoded, y_pred_classes)

0.9522218509856332